In [ ]:
#Importing all the necessary packages

import torch
import torchvision
import torchaudio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tqdm
import PIL
import sklearn
import streamlit

print("✅ All packages are imported successfully!")


In [ ]:
#Load and Preprocess Dataset 
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for ResNet50
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

# Define dataset paths
data_dir = r"D:\Projects\Dataset"

# Load datasets
train_dataset = datasets.ImageFolder(root=f"{data_dir}/Train", transform=transform)
val_dataset = datasets.ImageFolder(root=f"{data_dir}/Validation", transform=transform)
test_dataset = datasets.ImageFolder(root=f"{data_dir}/Test", transform=transform)

# Create DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print(f"✅ Training samples: {len(train_dataset)}")
print(f"✅ Validation samples: {len(val_dataset)}")
print(f"✅ Test samples: {len(test_dataset)}")

# Check class names
print(f"Classes: {train_dataset.classes}")


In [7]:
#Load and Modify ResNet50 Model

import torch
import torchvision.models as models
import torch.nn as nn

# Load the pretrained ResNet50 model
resnet50 = models.resnet50(pretrained=True)

# Modify the final fully connected layer for binary classification
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, 2)  # 2 classes: Real & Fake

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)

print("✅ ResNet50 model is ready for fine-tuning on Deepfake dataset!")


d:\Projects\Deepfake\deepfake_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Projects\Deepfake\deepfake_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\mahi4/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:09<00:00, 10.5MB/s]


✅ ResNet50 model is ready for fine-tuning on Deepfake dataset!


In [8]:
#Data Augmentation & DataLoader Setup
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define data transformations
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match ResNet50 input
        transforms.RandomHorizontalFlip(),  # Augmentation for better learning
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Define dataset directory
data_dir = "D:/Projects/Dataset"

# Load datasets
datasets_dict = {
    "train": datasets.ImageFolder(root=f"{data_dir}/Train", transform=data_transforms["train"]),
    "val": datasets.ImageFolder(root=f"{data_dir}/Validation", transform=data_transforms["val"]),
    "test": datasets.ImageFolder(root=f"{data_dir}/Test", transform=data_transforms["test"])
}

# Create DataLoaders
batch_size = 32  # Adjust batch size based on available GPU memory
dataloaders = {
    "train": DataLoader(datasets_dict["train"], batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True),
    "val": DataLoader(datasets_dict["val"], batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
    "test": DataLoader(datasets_dict["test"], batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
}

# Check dataset sizes
print(f"✅ Training samples: {len(datasets_dict['train'])}")
print(f"✅ Validation samples: {len(datasets_dict['val'])}")
print(f"✅ Test samples: {len(datasets_dict['test'])}")
print(f"Classes: {datasets_dict['train'].classes}")  # Should output ['Fake', 'Real']

✅ Training samples: 140002
✅ Validation samples: 39428
✅ Test samples: 10905
Classes: ['Fake', 'Real']


In [ ]:
#Feature Extraction 
import torch
import torch.nn as nn
import torchvision.models as models

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# Load pretrained ResNet50 model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Freeze all layers except the final classification layer
for param in model.parameters():
    param.requires_grad = False  # This prevents updating existing weights

# Modify the final layer for binary classification (2 classes: Fake, Real)
num_ftrs = model.fc.in_features  # Get the number of input features to the final layer
model.fc = nn.Linear(num_ftrs, 2)  # Change output layer to 2 classes

# Move the model to GPU if available
model = model.to(device)

print("✅ ResNet50 model is ready for training on Deepfake dataset!")


✅ Using device: cpu


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\mahi4/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:08<00:00, 11.8MB/s]


✅ ResNet50 model is ready for training on Deepfake dataset!
